In [1]:
# Imports
import cv2
import glob
import random
import pandas as pd
import statistics

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Taken from pretrained_example.py
import os
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator

# Off-the-shelf recognizer
import face_recognition

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageDraw, ImageFont

from scipy.stats import shapiro, normaltest

In [2]:
# Plot latent vectors of shape 18x512
def generate_image(latent_vector, size=512):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img.resize((size, size))

In [3]:
def setup():
    tflib.init_tf()
    # Load pre-trained network.
    url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
    with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
        _G, _D, Gs = pickle.load(f)
    generator = Generator(Gs, batch_size=1, randomize_noise=False) # -- RUNNING >1 TIMES THROWS ERROR
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    return [_G, _D, Gs, generator, fmt]

In [4]:
# Only run once.
[_G, _D, Gs, generator, fmt] = setup()

### Question #1 
Is the average/centroid of each subject's 4-tuple still recognizable as the same person?

In [5]:
subjects = [s.split('/')[-1].split('_')[0] for s in glob.glob('./generated_images/*_centroid.png')]

In [8]:
subjects[0]

'04549'

In [ ]:
# for subject in subjects:
#     print(subject)
#     lr_hr = glob.glob('./latent_representations/'+subject+'*_hr_*')
#     print(lr_hr)
#     lr_hr_c = np.load(lr_hr[0]) + np.load(lr_hr[1])
#     lat_rep_lr = glob.glob('./latent_representations/'+subject+'*_lr_*')

In [ ]:
# def get_match_rate(subject):
#     gi_fnames = glob.glob('./generated_images/'+subject+'*.png')
#     # gi_fnames
#     c_enc = None
#     gi_enc = []
#     for fname in gi_fnames:
#         img = np.array(Image.open(fname))
#         enc = face_recognition.face_encodings(img)
#         if len(enc)>0: enc = enc[0]
#         else: enc = np.zeros(128,)
#         if 'centroid' in fname: c_enc = enc
#         else: gi_enc.append(enc)
#     distances = face_recognition.compare_faces(gi_enc, c_enc)
#     return sum(distances)/4

In [ ]:
# match_rates = []
# for subject in subjects:
#     print(subject)
#     match_rates.append(get_match_rate(subject))

In [ ]:
# df_match_rates = pd.DataFrame(match_rates, index=subjects, columns=['match_rate'])
# df_match_rates['match_rate'].value_counts()
# df_match_rates.to_csv('df_match_rates.csv')

In [9]:
# Looks like most (190/192) of the subjects have centroids that are 
# recognizable when compared to the other images in the 4-tuple.
# What are the two exceptions to this:

df_match_rates = pd.read_csv('df_match_rates.csv')
df_match_rates[df_match_rates['match_rate']!=1.0]

# 04561 -- 0 out of 4 images matched against the centroid image's encoding.
# 04295 -- 2 out of 4 images matched against the centroid image's encoding.

,Unnamed: 0,match_rate
1,4561,0.0
28,4295,0.5


In [ ]:
# gi_0 = glob.glob('./generated_images/04561*')
# for fname in sorted(gi_0):
#     img = np.array(Image.open(fname))
#     plt.figure()
#     plt.title(fname.split('/')[-1])
#     plt.imshow(img)
#     plt.show()

In [ ]:
# gi_2 = glob.glob('./generated_images/04295*')
# for fname in sorted(gi_2):
#     img = np.array(Image.open(fname))
#     plt.figure()
#     plt.title(fname.split('/')[-1])
#     plt.imshow(img)
#     plt.show()

In [ ]:
# What does this mean? 
# We can probably sample from anywhere along any interpolating lines between centroid
# and one of the four images to get a new face of the same person.
# Furthermore, we can probably sample anywhere within the quadrilaterial created by 
# these interpolating lines.
# Let's test this out with PF.

#### Let's look at the centroids more in depth.

In [ ]:
ci_fnames = glob.glob('./generated_images/*_centroid.png')

In [ ]:
# ci_encs = []
# for ci_fname in ci_fnames:
#     img = Image.open(ci_fname)
#     enc = face_recognition.face_encodings(np.array(img))
#     if len(enc)>0: 
#         enc = enc[0]
#     else: 
#         print('No encoding found for '+ci_fname)
#         enc = np.zeros(128,)
#     ci_encs.append(enc)

In [ ]:
# with open('ci_encs.pkl', 'wb') as f:
#     pickle.dump(ci_encs, f)

In [ ]:
# with open('ci_encs.pkl', 'rb') as f:
#     ci_encs = pickle.load(f)

In [ ]:
ci_lr_fnames = glob.glob('./latent_representations/*_centroid.npy')
ci_lr_fnames[21:24]

In [ ]:
ci_lr_vecs = []
for ci_lr_fname in ci_lr_fnames:
    ci_lr_vecs.append(np.load(ci_lr_fname))

In [ ]:
len(ci_fnames), len(ci_encs), len(ci_lr_fnames), len(ci_lr_vecs)

In [ ]:
# ci_fnames.pop(1)
# ci_encs.pop(1) 
# ci_lr_fnames.pop(1) 
# ci_lr_vecs.pop(1)

In [ ]:
dist_ci_encs = []
match_ci_encs = []
for enc in ci_encs:
    dist_ci_encs.append(face_recognition.face_distance(ci_encs, enc))
    match_ci_encs.append(face_recognition.compare_faces(ci_encs, enc))

In [ ]:
dist_ci_lr_vecs = []
for vec in ci_lr_vecs:
    dist_ci_lr_vecs.append(np.linalg.norm(ci_lr_vecs - vec, axis=1))

In [ ]:
dist_ci_enc_25 = np.array([dist[0:25] for dist in dist_ci_encs[0:25]])

In [ ]:
match_ci_enc_25 = np.array([[int(b) for b in match_ci_encs[i][0:25]] for i in range(25)])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))

im1 = ax1.imshow(dist_ci_enc_25, interpolation='nearest', cmap='Blues')

ax1.set_title('face_recognition distance\nbetween subject centroid faces')
ax1.set_xlabel('Subject ID')
ax1.set_ylabel('Subject ID')

fig.colorbar(im1, ax=ax1)

im2 = ax2.imshow(match_ci_enc_25, cmap='binary', interpolation='nearest')

ax2.set_title('face_recognition binary match\nbetween subject centroid faces')
ax2.set_xlabel('Subject ID')
ax2.set_ylabel('Subject ID')

fig.colorbar(im2, ax=ax2)

plt.show()

In [ ]:
# Looks like there are some false positive matches... Let's see how many.
fp_record = {}
for i in range(len(match_ci_encs)):
    match_list = np.array(match_ci_encs[i])
    fp_record[i] = np.where(match_list==True)
fp_record

In [ ]:
len(dist_ci_lr_vecs)

In [ ]:
np.array(dist_ci_encs).shape, np.array(dist_ci_lr_vecs).shape

In [ ]:
# mean_fr_distances = df_fr_distances.describe().loc['mean']
# _ = plt.hist(mean_fr_distances, bins='auto')

In [ ]:
# def is_normal(data, alpha=0.05, method='shapiro'):
#     if method=='shapiro':
#         stat, p = shapiro(data)
#     else:
#         stat, p = normaltest(data)
#     print(stat, p)
#     if p > alpha:
#         print('Sample looks Gaussian (fail to reject H0)')
#     else:
#         print('Sample not look Gaussian (reject H0)')

In [ ]:
# is_normal(mean_fr_distances, alpha=0.05, method='dagostino')

In [ ]:
# sg_distances = []
# for i, centroid_lr_fname in enumerate(centroid_lr_fnames):   
#     #false_positives.append(sum(match) - 1)

In [ ]:
# import plotly.graph_objects as go
# fig = go.Figure(
#     data=[
#         go.Surface(z=distances_df.values),
#     ]
# )
# fig.update_layout(
#     title='Euclidean distance between centroid encodings', 
#     autosize=False,
#     width=500, 
#     height=500,
#     margin=dict(l=65, r=50, b=65, t=90))
# fig.show()

In [ ]:
# false_positives_df = pd.DataFrame(false_positives, index=centroids, columns=['fp'])

In [ ]:
# false_positives_df['fp'].value_counts()

In [ ]:
# _ = plt.hist(mean_fr_distances, bins='auto') 

In [ ]:
def interpolate_centroids(sub1, sub2, steps=60, pkl_fname='interpolate_centroids.pkl'):
    
#     ensemble_sub1 = subjects[sub1]
#     ensemble_sub2 = subjects[sub2]
    
#     centroid1 = ensemble_sub1['latent_representations_mean']
#     centroid2 = ensemble_sub2['latent_representations_mean']
    
#     centroid_img1 = generate_image(centroid1)
#     centroid_img2 = generate_image(centroid2)
    
    centroid_img1 = Image.open('./generated_images/'+sub1+'_centroid.png')
    centroid_img2 = Image.open('./generated_images/'+sub2+'_centroid.png')
    
    centroid1 = np.load('./latent_representations/'+sub1+'_centroid.png')
    centroid2 = np.load('./latent_representations/'+sub2+'_centroid.png')
    
    vec1_slim = np.reshape(centroid1, [1, centroid1.shape[0] * centroid1.shape[1]])
    vec2_slim = np.reshape(centroid2, [1, centroid2.shape[0] * centroid2.shape[1]])
    
    enc1 = face_recognition.face_encodings(np.array(centroid_img1))[0]
    enc2 = face_recognition.face_encodings(np.array(centroid_img2))[0]

    known_encodings = [enc1, enc2]
    
    images = []
    fr_distances = []
    
    z = np.empty((steps, vec1_slim.shape[1]))
    for i, alpha in enumerate(np.linspace(start=1.0, stop=0.0, num=steps)):

        # Linearly interpolate.
        z[i] = (alpha) * vec1_slim + (1.0-alpha) * vec2_slim

        # Reshape interpolated vector, and get interpolated image.
        curr_vec = np.reshape(z[i], [18, 512]) # back to original shape
        curr_img = np.array(generate_image(curr_vec))

        curr_enc = face_recognition.face_encodings(curr_img)[0]
        fr_distance = face_recognition.face_distance(known_encodings, curr_enc)

        images.append(np.array(curr_img))
        fr_distances.append(fr_distance)
        
    return_dict = {
        'images': images,
        'fr_distances': fr_distances,
    }
    
    return return_dict

In [ ]:
# results = interpolate_centroids('04261', '02463')

In [ ]:
# results['fr_distances']

In [ ]:
# idx_sub1 = 0
# idx_sub2 = 0
# for i, pair in enumerate(rd['fr_distances']):
#     if pair[0] < 0.4:
#         idx_sub1 = i
#     if pair[1] > 0.4:
#         idx_sub2 = i+1

In [ ]:
# idx_sub1, idx_sub2

In [ ]:
def outwards_from_centroid(subject):

    lr_fnames = glob.glob('./latent_representations/'+subject+'*')
    lr_fnames

    lr_c = None
    lr_gi = []
    for fname in lr_fnames:
        lr = np.load(fname)
        if 'centroid' in fname: lr_c = lr
        else: lr_gi.append(lr)
    # lr_fnames[0], lr_gi[0]

    for ep in range(4):
        
        print('Heading to endpoint: '+str(ep))
        center_slim = np.reshape(lr_c, [1, lr_c.shape[0] * lr_c.shape[1]])
        endpoint_slim = np.reshape(lr_gi[ep], [1, lr_c.shape[0] * lr_c.shape[1]]) # CHANGE 0 to i LATER
        
        center_img = Image.open('./generated_images/04261_centroid.png')
        endpoint_img = Image.open(
            './generated_images/'+lr_fnames[ep].split('/')[-1][:-3]+'png'
        )
        
        center_enc = face_recognition.face_encodings(np.array(center_img))
        if len(center_enc)>0: center_enc = center_enc[0]
        else: center_enc = np.zeros(128,)
        
        endpoint_enc = face_recognition.face_encodings(np.array(endpoint_img))
        if len(endpoint_enc)>0: endpoint_enc = endpoint_enc[0]
        else: endpoint_enc = np.zeros(128,)  
        
        known_encodings = [center_enc, endpoint_enc]
    
        images = []
        fr_distances = []

        steps = 30
        z = np.empty((steps, center_slim.shape[1]))
        for i, alpha in enumerate(np.linspace(start=1.0, stop=0.0, num=steps)):

            # Linearly interpolate.
            z[i] = (alpha) * center_slim + (1.0-alpha) * endpoint_slim

            # Reshape interpolated vector, and get interpolated image.
            curr_vec = np.reshape(z[i], [18, 512]) # back to original shape
            curr_img = np.array(generate_image(curr_vec))

            curr_enc = face_recognition.face_encodings(curr_img)
            if len(curr_enc)>0: curr_enc = curr_enc[0]
            else: curr_enc = np.zeros(128,)          
            fr_distance = face_recognition.face_distance(known_encodings, curr_enc)

            images.append(np.array(curr_img))
            fr_distances.append(fr_distance)
            
        s = images[0].shape
        steps = 30
        videowriter =  cv2.VideoWriter('./pf_endpoint'+str(ep)+'.avi', cv2.VideoWriter_fourcc(*'mp4v'), steps/10, (s[1], s[0]))
        for i in range(len(images)):
            videowriter.write(images[i][...,::-1])
        videowriter.release()   
            